In This Problem, We want to investigate the minimum vertex cover problem. Informally, A vertex cover is a subset of vertices that cover all the edges. i.e., for each edge, there exists an endpoint in the vertex cover. A minimum vertex cover is a vertex cover with the least amount of vertices possible.


The Formal definition of the problem is as follows:
A vertex cover $V'$ of an undirected graph $G = (V,E)$ is a subset of $V$ such that for all edge $uv \in E \Rightarrow u \in V' \vee v \in V'$. The vertex cover with the smallest possible size is called the minimum vertex cover.

Minimum vertex cover is a famous NP-Hard optimization problem. It means that we currently don't have any polynomial-time algorithm for this problem, and we will most likely never have such an algorithm unless $P = NP$. Therefore it is reasonable to use optimization algorithms like local search to find an approximate but not necessarily perfect answer.

In this question and the next one, you should implement two different local search techniques to solve this problem: Simulated Annealing for this question and the Genetic Algorithm for the next one.

In this problem, we provide you some of the functions to solve the problem. Some of them are complete, but some are incomplete, and you should complete them.

## Libraries

In [ ]:
import numpy as np
from math import exp
import random
import matplotlib.pyplot as plt
import math

## Graph Data
There is a file named **create_graph.py** beside this jupyter notebook. You can generate multiple types of networks by running the file using the syntax below. It is **not** mandatory and does not have any additional score. It is just for you to test your code to see whether it works or not.
```
python create_graph.py
```
Make sure that you have the dependencies installed, especially networkx version: 2.5.1+. You can use `pip`  to install the newest version of networkx using
```
pip install networkx
```
Also, you may want to change variables m2n and n in `create_graph.py`.

n is the number of nodes. m2n is roughly equal to the number of edges divided by the number of nodes.
 

## Reading graph data

We provide graph data in '.txt' files as an adjacency matrix. The first line indicates $n$, the number of vertices. The following $n$ lines each contain $n$ comma-separated numbers. Consider $d_{ij}$ to be the number in $i$th row and $j$th column. It is $1$ if and only if there is an edge between node $i$ and $j$, and $0$ otherwise. The function is provided completely for your comfort, and you don't need to change it.

In [ ]:
"""
No need to change this cell. You can change "./Inputs/test-q3-q4.txt" to test different graphs.
"""

graph_matrix =[]
def load_data(path = "./Inputs/test-q3-q4.txt"):
    with  open(path , 'r') as f:
        lines = f.readlines()
        number_of_vertices = int(lines[0])
        for i in range(number_of_vertices):
            line_split = lines[i+1].split(',');
            graph_matrix.append([])
            for j in range(number_of_vertices):
                graph_matrix[i].append(int(line_split[j]))
load_data()

## Edge Count

Complete the following line to find out the number of graphs edges from its adjacency matrix. It can be done in one line using NumPy.

In [ ]:
edge_count = np.sum(graph_matrix)//2
print(edge_count)

## State

To use local search algorithms, we must define our state-space. As the problem is about choosing a vertex or not, we define each state as a vector $S$ of size $n$. 

$$
S_i=   \left\{
\begin{array}{ll}
      0 & \text{vertex $v_i$ is not in vertex cover} \\
      1 & \text{vertex $v_i$ is in vertex cover}
\end{array} 
\right.
$$

Write a function that generates a random state. i.e., it must return a boolean vector of size $n$.

In [ ]:
def random_state_generator(n):
    v = np.random.randint(0, 2,(n))
    return v

We must also define the neighbor state. We can use multiple definitions for neighbor states, But here we use a simple one. Two states are neighbors if and only if they differ in just one element. Complete the following function, which gets a state and returns a new state by randomly choosing one element in the previous state and changing it from $0$ to $1$ or $1$ to $0$.

It is strongly recommended that you return the index of the changed element and its value, as you will likely need that later.

In [ ]:
def neighbour_state_generator(state):
    new_state = state.copy()
    n = len(state)
    vertex_to_change = np.random.randint(0, n)
    previous_value = state[vertex_to_change]
    new_state[vertex_to_change] = 1 -  state[vertex_to_change]
    return new_state, previous_value, vertex_to_change

## Cost Function

We must also define a cost function to evaluate each state. There are many ways to define a cost function for the vertex cover problem. Here we propose a cost function that works well with the problem. You could use other cost functions if you believe they work better. 

Our Proposed cost function is:

$C(s)=A*\sum_{i=1}^{n} s_{i}+ B*\sum_{i=1}^{n} \sum_{j=1}^{n} \mathrm{~d}_{i j} \overline{s_{i} \vee s_{j}}$

Here, $s_i$ is the $i\text{th}$ element of state vector. $\overline{s_{i} \vee s_{j}}$ is logical NOR of $v_i$ and $v_j$.

$A$ and $B$ are two arbitrary constants. Usually $A=1$ and $B=1$, but you could use other values.

Note that the formula above uses a 1-based index as a mathematical equation, but Python uses a 0-based index.

In [ ]:
def cost_function(graph_matrix,state , A = 1 , B=5):
    cost = 0;
    n = len(state)
    cost += A*np.sum(state)
    Nor_Mat = np.ones((n,n))
    for i in range(n):
        if (state[i] == 1): 
                Nor_Mat[i,:] = 0
                Nor_Mat[:,i] = 0
    cost += B*np.sum(Nor_Mat*graph_matrix)
    return cost

## Simulated Annealing

Now comes the central part. Simulated annealing is somehow like normal hill-climbing, but with a small but significant change. In each iteration of Simulated Annealing, we calculate the cost of this new state after generating a neighbor state. If it is better than the current state, we accept it and use it as the new current state. But if it is worse than the current state, we calculate an acceptance probability and accept it based on this probability.

Usually, $P = e^{-\frac{\Delta C}{T}}$ is used for acceptance probability, in which $\Delta C$ is the cost difference between the new state and previous state. $T$ is a positive parameter called Temperature. At first, it is initialized based on the problem domain. After each iteration, it is multiplied by a factor of $\alpha<1$ to make it smaller. The point of using $T$ is to allow more random perturbations at the start of the algorithm to prevent getting stuck in a local extremum.

For the vertex cover problem, there exist better acceptance probability equations. We consider one here, but you are free to use other equation.




### Vertex Cover Probability Acceptance Equation

Let $s$ be current state and $s'$ be a neighbour state that has worse cost function that $s$. Also suppose that only $i$th element is different between $s$ and $s'$. For calculating acceptance probability of $s'$ we use this equation: 


$$p= \begin{cases}\mathrm{e}^{-\frac{\Delta F\left(1-D e g\left(s_{i}\right)\right)}{T}} & s_{i}=1 \\ \mathrm{e}^{-\frac{\Delta F\left(1+D e g\left(s_{i}\right)\right)}{T}} & s_{i}=0\end{cases}$$
in which
$$\operatorname{Deg}\left({i}\right)=\frac{\operatorname{Degree}(i)}{E d g e N u m}$$

Note that $Deg(s_i)$ is actually independent of state and can be calculated beforehand for each vertex.

Complete the line bellow the make an array named 'deg', in which $i$th element corresponds to $Deg(i)$

In [ ]:
deg = np.sum(graph_matrix,axis=0)/edge_count

Now complete the function 'prob_accept' that gets current and next state costs and other related information and returns the probability of acceptance of the next state.

In [ ]:
def prob_accept(c0,c1,state,T,deg,vertex_to_change): 
    delat_c = c1-c0
    if state[vertex_to_change]==1:
        p = exp(-delat_c*(1-deg[vertex_to_change])/T)
    else:
        p = exp(-delat_c*(1+deg[vertex_to_change])/T)
    return p

Now Implement a function called 'accept'. This function gets the current state and next state and other required parameters as input and outputs True or False to identify whether we should accept the next state or not.

In [ ]:
def accept(current_state,next_state,graph_matrix,T,deg,vertex_to_change):
    c0 = cost_function(graph_matrix,current_state)
    c1 = cost_function(graph_matrix,next_state) 
    if c1 < c0 :
        return True
    else: 
        p = prob_accept(c0,c1,current_state,T,deg,vertex_to_change)
        res = np.random.binomial(1, p, 1)   
        if res == 1:
            return True
        return False

Now Implement the main 'anneal' function that performs the algorithm. Try to experiment with different initial T and $alpha$ values to get the best result. Also, your function must implement these two stop criteria:

- If the number of iterations exceeded 'stopping_iter', stop.
- If the temperature drops below 'stopping_temprature', stop.

Also, the cost_list array is defined before the function. Append the cost of the best solution found after each iteration to this array. Also, print out the best solution and the best cost obtained.

In [ ]:
cost_list = []


def anneal(
    graph_matrix, stopping_temperature=1e-8, stopping_iter=2000, alpha=0.999, T=70
):
    counter = 0
    n = np.size(graph_matrix,axis=0)
    current_state = random_state_generator(n)
    cost_list.append(cost_function(graph_matrix,current_state))
    edge_count = np.sum(graph_matrix)//2
    deg = np.sum(graph_matrix,axis=0)/edge_count
    while(1):
        state1 = T<=stopping_temperature
        state2 = counter>=stopping_iter
        if state1 or state2:
            break
        next_state, previous_value, vertex_to_change = neighbour_state_generator(current_state)
        res = accept(current_state,next_state,graph_matrix,T,deg,vertex_to_change)
        if res == True: 
            current_state = next_state
        T = T * 0.99
        counter += 1
        cost_list.append(cost_function(graph_matrix,current_state))
        print(counter,cost_list[-1])
    best_solution = current_state
    best_cost = cost_function(graph_matrix,current_state)
    print(T)
    print(counter)
    return best_solution, best_cost

In [ ]:
best_sol_SA, best_cost_SA = anneal(
    graph_matrix,
)

In [ ]:
best_sol_SA

Plot the cost vs. iteration graph.

In [ ]:
def plot_cost(cost_list):
    plt.plot(cost_list)
    plt.ylabel('Cost')
    plt.xlabel('Iteration')
    plt.grid(True)
    plt.title( 'Cost vs. Iteration Graph')
    pass

plot_cost(cost_list)